In [ ]:
from casadi import *

# 5.1 Unconstrained Optimization

On page 21 of their 2018 CasADi paper, we are working on minimizing **Rosenbrock's banana-valley function**.

\begin{equation*}
  \min_{x,y}
  ~~
  x^2
  +
  100
  \left(
    y - (1 - x)^2
  \right)^2
\end{equation*}

We use the nonlinear programming (NLP) solver called IPOPT, which takes functions $f(x, p)$, $g(x, p)$, decision variable $x$ and known parameter variable $p$.  Here we only need to specify $f(x)$ and $x$ since we have no constraints.

In [ ]:
x = SX.sym('x')
y = SX.sym('y')

In [ ]:
z = y - (1 - x)**2
f = x**2 + 100 * z**2
f

In [ ]:
P = {'x': vertcat(x, y), 'f': f}
F = nlpsol('F', 'ipopt', P)
F

That looks like some garbage, but it's ultimately a function that takes two $x$ parameters, two lower bounds for $x$ (which are $-\infty$ by default), two upper bounds for $x$ (which are $\infty$ by default), and the associated lambda variables for the Lagrangian dual.

Now to perform the optimization.

In [ ]:
r = F(x0=[2.5, 3.0])

In [ ]:
print(r['x'])
r

# 5.2 Nonlinear Programming Example

We now solve the same problem as in 5.1, but this time as a constrained optimization problem by _lifting_ it using a new variable $z$.

\begin{align*}
 \min_{x,y,z}
  & ~~
   x^2 + 100 z^2
 \\
 \text{subject to}
  & ~~
   z + (1 - x)^2 - y = 0
\end{align*}

This is the exact same problem, just framed with a constraint.  We can see this by solving the constraint for $z$ and substituting it in to the minimization.  However, that cannot always be done in general.

In [ ]:
x = SX.sym('x')
y = SX.sym('y')
z = SX.sym('z')
f = x**2 + 100 * z**2
g = z + (1-x)**2 - y
print('f: ', f)
print('g: ', g)

In [ ]:
P = {'x': vertcat(x, y, z), 'f': f, 'g': g}
F = nlpsol('F', 'ipopt', P)
F

In [ ]:
r = F(x0=[2.5, 3.0, -(1-2.5)**2 + 3.0], ubg=0, lbg=0)

In [ ]:
r

# 5.3 Automatic Sensitivity Analysis Example

Here we want to analyze the [Van der Pol oscillator](https://en.wikipedia.org/wiki/Van_der_Pol_oscillator) using the following ordinary differential equation (ODE):

\begin{equation*}
 \ddot{x}
  =
   (1 - x^2)
   \dot{x}
   -
   x
   +
   p,
 \quad
 x(0) = 1,
 ~
 \dot{x}(0) = 0
\end{equation*}

In order to solve this nonlinear ODE, we convert it into differential-algebraic equations (DAE) by renaming $x \rightarrow x_2$ and introducing a new variable $x_1 = \dot{x_2}$.

\begin{equation*}
 \left\{
  \begin{array}{ll}
   \dot{x}_1
    =
     (1 - x_2^2)
     x_1
     -
     x_2
     +
     p,
    & \quad
    x_1(0) = 0
   \\
   \dot{x}_2
    =
     x_1,
    & \quad
    x_2(0) = 1
  \end{array}
 \right.
\end{equation*}

Here we will fix $p = 0.1$ and solve for $x_f = x(1)$.

In [ ]:
x = SX.sym('x', 2) # x is of length 2
p = SX.sym('p')
z = 1 - x[1]**2
f = vertcat(z*x[0] - x[1] + p, x[0])
dae = {'x': x, 'p': p, 'ode': f}
dae

In [ ]:
F = integrator('F', 'cvodes', dae, {'t0': 0, 'tf': 1})
F

In [ ]:
r = F(x0=[0, 1], p = 0.1)

In [ ]:
r

Now to do the sensativity analysis

In [ ]:
D = F.factory('D', ['x0', 'p'], ['jac:xf:x0'])
D

In [ ]:
r = D(x0=[0, 1], p = 0.1)
r

I'm not sure how to interpret such a result.  I have not investigated what their sensativity analysis is doing, returning, or how to interpret the results.  Perhaps this is something worth looking into further.

# 5.4 Direct Multiple Shooting Method

I did not investigate further the direct multiple shooting method, but let's follow the steps below and do it.

We take again the Van der Pol oscillator, but replace $p$ with a control variable $u(t)$ in order to minimize movement, velocity, and control.  This is now turned into an optimal control problem (OCP).

\begin{align*}
 \min_{x(\cdot), z(\cdot), u(\cdot)}
  & \quad
 \int_0^T
 \left(
  x_1(t)^2
  +
  x_2(t)^2
  +
  u(t)^2
 \right)
 dt
 \\
 \text{subject to}
  & \quad
 \left\{
  \begin{array}{l}
   \dot{x}_1(t)
    =
     z(t) x_1(t)
     -
     x_2(t)
     +
     u(t)
   \\
   \dot{x}_2(t)
    =
     x_1(t)
   \\
   0
    =
     x_2(t)^2 + z(t) - 1
   \\
   -1.0 \leq u(t) \leq 1.0,
   \quad
   x_1(t) \geq -0.25
  \end{array}
 \right.
 \\
 & \quad
  \text{for }
  t \in [0, T]
 \\
 & \quad
 x_1(0) = 0,
 \quad
 x_2(0) = 1
\end{align*}

We want to convert this into a nonlinear program (NLP) format so that we can enter it into CasADi.

\begin{align*}
 \min_{x, p}
  & \quad
   f(x, p)
 \\
 \text{subject to}
  & \quad
   \underline{x}
   \leq
   x
   \leq
   \overline{x},
   \quad
   p = \underline{\overline{p}},
   \quad
   \underline{g}
   \leq
   g(x, p)
   \leq
   \overline{g}
\end{align*}

Instead of continuous space, we discretize the space into $N$ time steps with piecewise constant control.

In [ ]:
x = SX.sym('x', 2)
z = SX.sym('z')
u = SX.sym('u')
f = vertcat(z*x[0] - x[1] + u, x[0])
g = x[1]**2 + z - 1
h = x[0]**2 + x[1]**2 + u**2
dae = {'x': x, 'p': u, 'ode': f, 'z': z, 'alg': g, 'quad': h}
dae

In [ ]:
T = 10 # end time
N = 20 # how many time steps
F = integrator('F', 'idas', dae, {'t0': 0, 'tf': T/N})
F

Now we construct a symbolic representation of the NLP.  For the formulation, we remove $p$, and set $G(x) = 0$.

\begin{align*}
 \min_{w}
  & \quad
   J(w)
 \\
 \text{subject to}
  & \quad
   G(w) = 0,
   \quad
   \underline{w}
   \leq
   w
   \leq
   \overline{w}
\end{align*}

In [ ]:
# Empty NLP
w = []
lbw = [] # lower bound for w
ubw = [] # upper bound for w
G = []
J = 0

In [ ]:
# Initial conditions
Xk = MX.sym('X0', 2)
w += [Xk]
lbw += [0, 1]
ubw += [0, 1]

In [ ]:
for k in range(1, N+1):
    # Local control
    Uk = MX.sym('U{}'.format(k-1))
    w += [Uk]
    lbw += [-1]
    ubw += [1]

    # Call the integrator
    Fk = F(x0=Xk, p=Uk)
    J += Fk['qf']

    # New local state
    Xk = MX.sym('X{}'.format(k), 2)
    w += [Xk]
    lbw += [-.25, -inf]
    ubw += [inf, inf]

    # Continuity constraint
    G += [Fk['xf'] - Xk]

In [ ]:
list(zip(lbw, w, ubw))

In [ ]:
G

In [ ]:
# Create the NLP solver
nlp = {'f': J, 'g': vertcat(*G), 'x': vertcat(*w)}
S = nlpsol('S', 'ipopt', nlp)
S

In [ ]:
# Solve the NLP
# Note: the x0, lbg, and ubg will be copied to be the same
#       length as 'x' and 'G' respectively
r = S(lbx=lbw, ubx=ubw, x0=0, lbg=0, ubg=0)

In [ ]:
r

In [ ]:
wsolved = r['x'].toarray()
x1solved = wsolved[0::3]
x2solved = wsolved[1::3]
usolved = wsolved[2::3]

In [ ]:
x1solved

In [ ]:
x2solved

In [ ]:
usolved